In [1]:
from dask.distributed import Client
import xarray as xr
import gcsfs
from sgkit_bgen.bgen_reader import unpack_variables
xr.set_options(display_style='text')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:8086 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
fs = gcsfs.GCSFileSystem()
store = gcsfs.mapping.GCSMap('rs-ukb/prep-data/gt-imputation/ukb_chrXY.zarr', gcs=fs, check=True, create=False)
ds = xr.open_zarr(store)
ds = unpack_variables(ds, dtype='float16')
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [3]:
cr = ds.call_genotype_probability_mask.mean(dim='samples').compute()
cr.max()

<xarray.DataArray 'call_genotype_probability_mask' ()>
array(0.)

In [6]:
import numpy as np
import dask.array as da
from numba import guvectorize
from sgkit.typing import ArrayLike
from xarray import Dataset, DataArray

@guvectorize(
    [
        "void(float32[:], uint8[:], uint8[:])",
    ],
    "(k),(n)->(n)",
    nopython=True,
)
def _hard_calls(gp: ArrayLike, _: ArrayLike, out: ArrayLike) -> None:
    out[:] = 0
    n_genotype = len(gp)
    i = np.argmax(gp)
    if i == 1:
        out[1] = 1
    else:
        out[:] = 1
        
# def hard_calls(ds: Dataset) -> DataArray:
#     ploidy: int = 2
#     n_genotypes = ds.dims["genotypes"]
#     G = da.asarray(ds["call_genotype_probability"])
#     shape = (G.chunks[0], G.chunks[1], ploidy)
#     N = da.empty(ploidy, dtype=np.uint8)
#     return xr.DataArray(
#         da.map_blocks(_hard_calls, G, N, chunks=shape, drop_axis=2, new_axis=2),
#         dims=("variants", "samples", "ploidy"),
#         name="call_genotype",
#     )

def hard_calls(ds: Dataset) -> DataArray:
    I = ds.call_genotype_probability.argmax(dim='genotypes').astype('uint8')
    return xr.concat([
        xr.where(I == np.uint8(0), np.uint8(0), np.uint8(1)),
        xr.where(I == np.uint8(2), np.uint8(1), np.uint8(0)),
    ], dim='ploidy').transpose('variants', 'samples', 'ploidy')

In [7]:
ds['call_genotype'] = hard_calls(ds)
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, ploidy: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, ploidy, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_genotype                   (variants, samples, ploidy) uint8 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [6]:
ds['call_genotype'][:10, :10, 0].compute()

<xarray.DataArray 'call_genotype' (variants: 10, samples: 10)>
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)
Dimensions without coordinates: variants, samples

In [8]:
ds['call_genotype_mask'] = ds.call_dosage_mask.broadcast_like(ds.call_genotype).astype(bool)
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, ploidy: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, ploidy, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_genotype                   (variants, samples, ploidy) uint8 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_genotype_mask              (variants, samples, ploidy) bool dask.array<chunksize=(10432, 11313, 2), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [10]:
import sgkit
ds['variant_hwe_p_value'] = sgkit.hardy_weinberg_test(ds)['variant_hwe_p_value']
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, ploidy: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, ploidy, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_genotype                   (variants, samples, ploidy) uint8 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_genotype_mask              (variants, samples, ploidy) bool dask.array<chunksize=(10432, 11313, 2), meta=np.ndarray>
    variant_hwe_p_value             (variants) float64 dask.array<chunksize=(10432,), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [ ]:
# This requires about 12GiB per core
p = ds['variant_hwe_p_value'].compute()
p